## Lab 13.a -- Crime Stats

Please ensure the first few code blocks execute in your notebooks.  Your lab tasks appear towards the end of the notebook.


In [6]:
import json
import os

print(os.getcwd())

## please load the data from the mp248-course-notes repository.
## It is a large file.  i.e. : Please **DO NOT** store in your personal repo, i.e.
## please do not copy this file, nor move this file.

## the code below should execute in your notebook -- if not, modify the directory
## accordingly.

with open("../data/Victoria (BC) Police Department.geojson") as f:
    data = json.loads(f.read())
    

/home/rybu/teaching/math248/mp248-course-notes/Labs


In [4]:
## This is our code from class to load the data. 
## Please ensure it executes.

import collections as co
import pprint as pp

ctree = dict()

for R in data['features']:
    TopI = R['properties']['parent_incident_type']
    BotI = R['properties']['incident_type_primary']
    if TopI not in ctree.keys():
        ctree[TopI] = co.defaultdict(int)
    ctree[TopI][BotI] += 1
    
pp.pprint(ctree)


{'Alarm': defaultdict(<class 'int'>, {'FALSE ALARMS': 1447}),
 'Arson': defaultdict(<class 'int'>,
                      {'ARSON-DISREGARD FOR HUMAN LIFE': 22,
                       'ARSON-PROPERTY': 260}),
 'Assault': defaultdict(<class 'int'>,
                        {'ASSAULT AGAINST POLICE OFFICER': 417,
                         'ASSAULT-COMMON OR TRESPASS': 2065,
                         'ASSAULTS-OTHER': 473}),
 'Assault with Deadly Weapon': defaultdict(<class 'int'>,
                                           {'ASSAULT OT P/O-W/WPN OR CBH': 38,
                                            'ASSAULT POLICE-W/WEAPON OR CBH': 75,
                                            'ASSAULT-AGGRAVATED': 98,
                                            'ASSAULT-OTHER PEACE OFFICER': 125,
                                            'ASSAULT-W/WEAPON OR CBH': 1552}),
 'Breaking & Entering': defaultdict(<class 'int'>,
                                    {'BREAK & ENTER-BUSINESS': 1691,
          

In [5]:
import datetime as dt

clean_dat = list()
incompl_dat = list()

for x in data['features']:
    if 'properties' not in x.keys():
        continue ## why bother?

    ## we have the properties, so let's start building the record.
    newRec = dict()
    incFlag = False # set to true if we discover missing data in x
    
    floatkeys = ['latitude', 'longitude']
    strkeys = ['parent_incident_type', 'incident_type_primary', 'address_1', 'address_2']
    intkeys = ['case_number', 'incident_id']
    datekeys = ['created_at', 'updated_at', 'incident_datetime']
    
    for k in floatkeys:
        if k in x['properties'].keys() and isinstance(x['properties'][k], str) and\
        len(x['properties'][k])>0:
            newRec[k] = float(x['properties'][k])
        else:
            incFlag=True

    ## anything to check?
    for k in strkeys:
        newRec[k] = x['properties'][k]
        
    for k in intkeys:
        ## let's count non-numerical characters
        non_ch_chars = []
        digits = '0123456789'
        for ch in x['properties'][k]:
            if ch not in digits:
                non_ch_chars.append(ch)

        for ch in non_ch_chars:
            if ch != '-':
                x['properties'][k] = x['properties'][k].replace(ch, '-')
            
        if '-' in x['properties'][k]:
            newRec[k] = [int(w) for w in x['properties'][k].split('-')]
        else:
            newRec[k] = int(x['properties'][k])
            
    for k in datekeys:
        ## typical format: '2014-12-06T21:59:00.000'
        newRec[k] = dt.datetime.strptime(x['properties'][k], '%Y-%m-%dT%H:%M:%S.000')
        
    if incFlag:
        incompl_dat.append(newRec)
    else:
        clean_dat.append(newRec)


**Problem 1:** As a sanity check on the data, use **matplotlib** to plot the GPS coordinates of all the data in the list **clean_dat**.  
(a) Does your plot look roughly like Victoria?   

(b) If there are some **outlier** data, investigate it.  Do the GPS coordinates correspond to the listed address? 

(c) Once you have purged the outliers from your data, your plot should look roughly like Victoria.  If not, what else is included in the data?


**Problem 2:** One would expect that the *incident_datetime* is when the reported
crime occured, so *created_at* would happen after the *incident_datetime*, and *updated_at*
would perhaps be the same as *created_at*, or further in the future.  Is this true for all the
records?  Count the number of cases where the data disagrees with the expected order.   What do you
think is the cause of this bad data?  
 

**Note**: We will use the outcome of this lab in the next lecture, and in our follow-up lab. 